In [17]:
import cv2
import numpy as np
import imutils

# Constants for chili detection
CHILI_MIN_ASPECT_RATIO = 0.2  
CHILI_MAX_ASPECT_RATIO = 5.0
CHILI_MIN_AREA = 2000
AFFECTED_AREA_THRESHOLD = 0.2  # Threshold for determining bad quality

def resize_image(image, width, height):
    """Resize image for display purposes."""
    return cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)

def detect_chili_quality(image_path):
    """Detect chili quality, calculate dimensions, and grade the results."""
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Input image not found or failed to load.")
        return
    print("Input image loaded successfully.")

    # Display original image
    cv2.imshow("Original Image", resize_image(image, 800, 600))

    alpha = 2.0
    beta = 0
    adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    cv2.imshow("Brightness and Contrast Adjusted", resize_image(adjusted_image, 800, 600))

    hsv = cv2.cvtColor(adjusted_image, cv2.COLOR_BGR2HSV)

    lower_red1 = np.array([0, 30, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 30, 50])
    upper_red2 = np.array([180, 255, 255])
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    chili_mask = mask1 + mask2
    cv2.imshow("HSV Mask", resize_image(chili_mask, 800, 600))

    kernel = np.ones((5, 5), np.uint8)
    chili_mask = cv2.morphologyEx(chili_mask, cv2.MORPH_CLOSE, kernel)
    chili_mask = cv2.morphologyEx(chili_mask, cv2.MORPH_OPEN, kernel)
    chili_mask = cv2.dilate(chili_mask, None, iterations=1)
    cv2.imshow("Morphological Cleaned Mask", resize_image(chili_mask, 800, 600))

    cnts = cv2.findContours(chili_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    output_image = adjusted_image.copy()

    lower_affected = np.array([5, 30, 90])
    upper_affected = np.array([50, 180, 270])

    mask_affected = cv2.inRange(hsv, lower_affected, upper_affected)
    cv2.imshow("Affected Area HSV Mask", resize_image(mask_affected, 800, 600))

    # Grading counters
    total_chilies = 0
    bad_quality_chilies = 0

    for i, c in enumerate(cnts):
        area = cv2.contourArea(c)
        if area > CHILI_MIN_AREA:
            x, y, w, h = cv2.boundingRect(c)
            aspect_ratio = float(h) / w if w > 0 else 0

            if CHILI_MIN_ASPECT_RATIO <= aspect_ratio <= CHILI_MAX_ASPECT_RATIO:
                total_chilies += 1

                chili_length = h
                chili_width = w

                single_chili_mask = np.zeros(chili_mask.shape, dtype=np.uint8)
                cv2.drawContours(single_chili_mask, [c], -1, 255, thickness=cv2.FILLED)

                affected_chili = cv2.bitwise_and(mask_affected, mask_affected, mask=single_chili_mask)

                cnts_affected = cv2.findContours(affected_chili, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                cnts_affected = imutils.grab_contours(cnts_affected)

                # Draw yellow contours on the affected regions
                for affected_contour in cnts_affected:
                    cv2.drawContours(output_image, [affected_contour], -1, (0, 255, 255), 2)

                # Calculate affected area ratio
                affected_area = cv2.countNonZero(affected_chili)
                affected_ratio = affected_area / area if area > 0 else 0

                # Label and color based on quality
                if affected_ratio > AFFECTED_AREA_THRESHOLD:
                    bad_quality_chilies += 1
                    label = "Bad Quality Chili"
                    color = (0, 0, 255)
                else:
                    label = "Good Quality Chili"
                    color = (0, 255, 0)

                cv2.rectangle(output_image, (x, y), (x + w, y + h), color, 2)
                cv2.putText(output_image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 2, color, 2)
                cv2.putText(output_image, f"Length: {round(chili_length*0.0264583333)}cm", 
                            (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 2)
                cv2.putText(output_image, f"Width: {round(chili_width*0.0264583333)}cm", 
                            (x, y + h +70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 2)

    # Final Grading Summary
    print(f"Total Chilies: {total_chilies}")
    print(f"Bad Quality Chilies: {bad_quality_chilies}")
    print(f"Good Quality Chilies: {total_chilies - bad_quality_chilies}")

    # Display grading results on the output image
    grading_text = f"Total: {total_chilies}, Bad: {bad_quality_chilies}, Good: {total_chilies - bad_quality_chilies}"
    cv2.putText(output_image, grading_text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)

    # Display final output
    cv2.imshow("Final Output", resize_image(output_image, 800, 600))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Path to the input image
image_path = r'C:\Users\vidya\OneDrive\Documents\Spiceup-SCM\Approch-2\Chilly\IMG_20241102_122_14.jpg'
detect_chili_quality(image_path)


Input image loaded successfully.
Total Chilies: 6
Bad Quality Chilies: 0
Good Quality Chilies: 6
